In [ ]:
import pandas as pd
import pymongo
import random
import datetime

from faker import Faker
from pymongo import MongoClient

fake = Faker('pt_BR')

mongo_url = ''
client = MongoClient(mongo_url)
db = client.heroku_l576fn73
db_students = db.students

In [ ]:
school_subjects = [
    { 'title': 'Fundamentos de Programação', 'semester': 1 },
    { 'title': 'Ferramentas Web e UX', 'semester': 1 },
    { 'title': 'Sistema Operacional', 'semester': 1 },
    { 'title': 'Soft Skills I', 'semester': 1 },
    { 'title': 'Teste de Software', 'semester': 2 },
    { 'title': 'Redes de Computadores', 'semester': 2 },
    { 'title': 'Soft Skills II', 'semester': 2 },
    { 'title': 'Requisitos, Modelagem e Análise de Dados', 'semester': 2 },
    { 'title': 'Programação Server-Side', 'semester': 2 },
    { 'title': 'Projeto e Arquitetura de Software', 'semester': 3 },
    { 'title': 'Estrutura de Dados', 'semester': 3 },
    { 'title': 'Empreendedorismo, Criatividade e Inovação', 'semester': 3 },
    { 'title': 'Banco de Dados', 'semester': 3 },
    { 'title': 'Programação Orientada a Objetos', 'semester': 3 },
    { 'title': 'Desenvolvimento de Dispositivos Móveis', 'semester': 4 },
    { 'title': 'Matemática Computacional', 'semester': 4 },
    { 'title': 'Soft Skills III', 'semester': 4 },
    { 'title': 'Gestão de Projetos', 'semester': 4 },
    { 'title': 'Qualidade de Software', 'semester': 4 },
    { 'title': 'Gerenciamento, Configuração e Processos de Software', 'semester': 5 },
    { 'title': 'Probabilidade e Estatística', 'semester': 5 },
    { 'title': 'Soft Skills IV', 'semester': 5 },
    { 'title': 'Segurança da Informação', 'semester': 5 },
    { 'title': 'Programação Web', 'semester': 5 },
    { 'title': 'Análise, Projeto e Desenvolvimento Ágil', 'semester': 6 },
    { 'title': 'Inteligência Artificial', 'semester': 6 },
    { 'title': 'Internet das Coisas', 'semester': 6 },
    { 'title': 'Sociedade Contemporânea', 'semester': 6 },
    { 'title': 'Manutenção e Melhoria de Software', 'semester': 6 },
    { 'title': 'Algoritmos Avançados', 'semester': 7 },
    { 'title': 'Pesquisa Operacional', 'semester': 7 },
    { 'title': 'Aprendizagem de Máquina', 'semester': 7 },
    { 'title': 'Disciplina Optativa', 'semester': 7 },
    { 'title': 'Governança de TI', 'semester': 8 },
    { 'title': 'Portifólio de Projeto', 'semester': 8 },
    { 'title': 'Análise Preditiva', 'semester': 8 },
    { 'title': 'Ciência de Dados', 'semester': 8  }
]

teachers = [
    'Glauco Vinícius Scheffel',
    'Rodrigo Mello Mattos Habib Gregori',
    'Dani Prestini',
    'Guilherme Defreitas Juraszek',
    'Alan Rafael Fachini',
    'Diogo Vinícius Winck',
    'Manfred Heil Junior',
    'Mauricio Henning'
]
        
for school_subject in school_subjects:
    school_subject['teacher'] = random.choice(teachers)
    
students = list()
quantity = 500

In [ ]:
for i in range(quantity):
    student = {}
    student['name'] = {}
    student['demographics'] = {}

    if (i % 2) == 0:
        student['name']['firstName'] = fake.first_name_male()
        student['name']['lastName'] = fake.last_name_male()
        student['demographics']['gender'] = 'M'
    else:
        student['name']['firstName'] = fake.first_name_female()
        student['name']['lastName'] = fake.last_name_female()
        student['demographics']['gender'] = 'F'
    
    students.append(student)
    

In [ ]:
for (student) in students:
    student['addresses'] = list()
    address = {}

    address['city'] = fake.city()
    address['neighborhood'] = fake.neighborhood()
    address['number'] = fake.building_number()
    address['postalCode'] = fake.postcode()
    address['state'] = fake.estado_sigla()
    address['street'] = fake.street_name()
    
    first_name = student['name']['firstName'].split()
    last_name = student['name']['lastName'].split()
    first_name = first_name[0].lower()
    last_name = last_name[len(last_name) - 1].lower()
    
    student['studentUsername'] = first_name + '.' + last_name
    student['email'] = student['studentUsername'] + '@' + fake.free_email_domain()
    student['currentSemester'] = random.randrange(1, 9, 1)
    
    birth_date = fake.date_of_birth(tzinfo=None, minimum_age=15, maximum_age=115)
    
    student['demographics']['projectedGraduationYear'] = fake.year()
    student['demographics']['birthDate'] = str(birth_date)
    student['addresses'].append(address)

In [ ]:
for student in students:
    student['schoolSubjects'] = list()
    
    for school_subject in school_subjects:
        student_school_subject = school_subject
        
        if (student['currentSemester'] >= student_school_subject['semester']):
            student_school_subject['grades'] = list()
            
            for i in range(3):
                grade = {}
                grade['description'] = 'Nota ' + str(i + 1)
                grade['grade'] = random.randrange(0, 11, 1)
                
                student_school_subject['grades'].append(grade)
            
            student['schoolSubjects'].append(student_school_subject)

In [ ]:
db_students.drop()
db_students.insert_many(students)
db_inserted_documents = db_students.find()

dataframe = pd.DataFrame(list(db_inserted_documents))
dataframe = dataframe.set_index("_id")
dataframe